# Creando con los Modelos de Mistral

## Introducción

En esta lección veremos:
- Exploración de los diferentes modelos de Mistral
- Comprender los casos de uso y escenarios para cada modelo
- Ejemplos de código que muestran las características únicas de cada modelo.


## Los modelos Mistral

En esta lección, exploraremos 3 modelos diferentes de Mistral: **Mistral Large**, **Mistral Small** y **Mistral Nemo**.

Cada uno de estos modelos está disponible de forma gratuita en el marketplace de modelos de Github. El código en este cuaderno usará estos modelos para ejecutar el código. Aquí tienes más detalles sobre cómo usar los modelos de Github para [prototipar con modelos de IA](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst).


## Mistral Large 2 (2407)
Mistral Large 2 es actualmente el modelo insignia de Mistral y está diseñado para uso empresarial.

El modelo es una mejora respecto al Mistral Large original, ofreciendo:
- Ventana de contexto más grande: 128k frente a 32k
- Mejor rendimiento en tareas de matemáticas y programación: 76.9% de precisión promedio frente a 60.4%
- Mayor rendimiento multilingüe: los idiomas incluyen inglés, francés, alemán, español, italiano, portugués, neerlandés, ruso, chino, japonés, coreano, árabe e hindi.

Con estas características, Mistral Large destaca en:
- *Generación aumentada por recuperación (RAG)* - gracias a la ventana de contexto más amplia
- *Llamadas a funciones* - este modelo tiene llamadas a funciones nativas que permiten la integración con herramientas externas y APIs. Estas llamadas pueden hacerse tanto en paralelo como de forma secuencial, una tras otra.
- *Generación de código* - este modelo sobresale en la generación de Python, Java, TypeScript y C++.


En este ejemplo, estamos usando Mistral Large 2 para ejecutar un patrón RAG sobre un documento de texto. La pregunta está escrita en coreano y consulta sobre las actividades del autor antes de la universidad.

Se utiliza el modelo de Cohere Embeddings para crear embeddings tanto del documento de texto como de la pregunta. Para este ejemplo, se usa el paquete de Python faiss como almacén de vectores.

El prompt enviado al modelo Mistral incluye tanto las preguntas como los fragmentos recuperados que son similares a la pregunta. Luego, el modelo proporciona una respuesta en lenguaje natural.


In [50]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [51]:
import requests
import numpy as np
import faiss
import os

from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
from azure.ai.inference import EmbeddingsClient

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
text = response.text

chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
len(chunks)

embed_model_name = "cohere-embed-v3-multilingual" 

embed_client = EmbeddingsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(token)
)

embed_response = embed_client.embed(
    input=chunks,
    model=embed_model_name
)



text_embeddings = []
for item in embed_response.data:
    length = len(item.embedding)
    text_embeddings.append(item.embedding)
text_embeddings = np.array(text_embeddings)


d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

question = "저자가 대학에 오기 전에 주로 했던 두 가지 일은 무엇이었나요?？"

question_embedding = embed_client.embed(
    input=[question],
    model=embed_model_name
)

question_embeddings = np.array(question_embedding.data[0].embedding)


D, I = index.search(question_embeddings.reshape(1, -1), k=2) # distance, index
retrieved_chunks = [chunks[i] for i in I.tolist()[0]]

prompt = f"""
Context information is below.
---------------------
{retrieved_chunks}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""


chat_response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content=prompt),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(chat_response.choices[0].message.content)

The author primarily engaged in two activities before college: writing and programming. In terms of writing, they wrote short stories, albeit not very good ones, with minimal plot and characters expressing strong feelings. For programming, they started writing programs on the IBM 1401 used for data processing during their 9th grade, at the age of 13 or 14. They used an early version of Fortran and typed programs on punch cards, later loading them into the card reader to run the program.


## Mistral Small 
Mistral Small es otro modelo de la familia Mistral, dentro de la categoría premier/empresarial. Como su nombre lo indica, este modelo es un Small Language Model (SLM). Las ventajas de usar Mistral Small son:
- Ahorro de costos en comparación con los LLMs de Mistral como Mistral Large y NeMo - reducción de precio del 80%
- Baja latencia - respuestas más rápidas en comparación con los LLMs de Mistral
- Flexible - se puede implementar en diferentes entornos con menos restricciones en los recursos necesarios.

Mistral Small es ideal para:
- Tareas basadas en texto como resumen, análisis de sentimiento y traducción.
- Aplicaciones donde se realizan solicitudes frecuentes gracias a su rentabilidad
- Tareas de código de baja latencia como revisión y sugerencias de código


## Comparando Mistral Small y Mistral Large

Para mostrar las diferencias de latencia entre Mistral Small y Large, ejecuta las siguientes celdas.

Deberías notar una diferencia en los tiempos de respuesta de entre 3 y 5 segundos. También observa las diferencias en la longitud y el estilo de las respuestas usando el mismo prompt.


In [ ]:
import os 
endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-small"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

## Mistral NeMo

En comparación con los otros dos modelos mencionados en esta lección, Mistral NeMo es el único modelo gratuito con licencia Apache2.

Se considera una mejora respecto al anterior LLM de código abierto de Mistral, Mistral 7B.

Otras características del modelo NeMo son:

- *Tokenización más eficiente:* Este modelo utiliza el tokenizador Tekken en lugar del más común tiktoken. Esto permite un mejor rendimiento en más idiomas y código.

- *Ajuste fino:* El modelo base está disponible para ajuste fino. Esto brinda mayor flexibilidad para casos de uso donde se requiera personalización.

- *Llamadas a funciones nativas* - Al igual que Mistral Large, este modelo ha sido entrenado para realizar llamadas a funciones. Esto lo hace único, ya que es uno de los primeros modelos de código abierto en ofrecer esta capacidad.


## Mistral NeMo

En comparación con los otros dos modelos mencionados en esta lección, Mistral NeMo es el único modelo gratuito con licencia Apache2.

Se considera una mejora respecto al anterior LLM de código abierto de Mistral, Mistral 7B.

Otras características del modelo NeMo son:

- *Tokenización más eficiente:* Este modelo utiliza el tokenizador Tekken en lugar del más común tiktoken. Esto permite un mejor rendimiento en más idiomas y código.

- *Ajuste fino:* El modelo base está disponible para ajuste fino. Esto brinda mayor flexibilidad para casos de uso donde se requiera personalización.

- *Llamadas nativas a funciones* - Al igual que Mistral Large, este modelo ha sido entrenado para realizar llamadas a funciones. Esto lo hace único, ya que es uno de los primeros modelos de código abierto en ofrecer esta capacidad.


### Comparando los tokenizadores

En este ejemplo, veremos cómo Mistral NeMo maneja la tokenización en comparación con Mistral Large.

Ambos ejemplos usan el mismo prompt, pero deberías notar que NeMo devuelve menos tokens que Mistral Large.


In [11]:
pip install mistral-common

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 kB 15.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "open-mistral-nemo	"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

128


In [13]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "mistral-large-latest"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

135


## El aprendizaje no termina aquí, continúa el viaje

Después de completar esta lección, visita nuestra [colección de aprendizaje sobre IA Generativa](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) para seguir ampliando tus conocimientos sobre IA Generativa.



---

**Descargo de responsabilidad**:  
Este documento ha sido traducido utilizando el servicio de traducción automática [Co-op Translator](https://github.com/Azure/co-op-translator). Si bien nos esforzamos por lograr precisión, tenga en cuenta que las traducciones automáticas pueden contener errores o inexactitudes. El documento original en su idioma nativo debe considerarse la fuente autorizada. Para información crítica, se recomienda una traducción profesional realizada por humanos. No nos hacemos responsables de malentendidos o interpretaciones erróneas que surjan del uso de esta traducción.
